# Example with a linear difference system

From the update (dalgebra-0.0.20220428.1815) we can treat now difference rings too. Let's see how to work with them with a very simple linear system:

In [8]:
%display latex
from dalgebra import *

The system has 2 state variables $C(n)$ and $S(n)$ and the difference system can be written using the parameters $K_{se}, I_0, K_{cp}$ and $K_{rb}$. We need to create these variables and set the difference ring such that everything is a constant:

$$\begin{array}{rcl}
    I_0 & \mapsto & \texttt{I_0}\\
    K_{se} & \mapsto & \texttt{K_se}\\
    K_{cp} & \mapsto & \texttt{K_cp}\\
    K_{rb} & \mapsto & \texttt{K_rb}
\end{array}$$

In [9]:
R.<I_0, K_se, K_cp, K_rb, K_sc> = QQ[]
DR = DifferenceRing(R)
## We update the variables
I_0, K_se, K_cp, K_rb, K_sc = DR.gens()
## We check that all these are constants
print("All are constants -> ", all(el.difference() == el for el in DR.gens()))

DR

All are constants ->  True


Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In order to build the system, we need to create the two difference variables $C(n)$ and $S(n)$. In the code, these will be represented by $\texttt{C}$ and $\texttt{S}$ such that $\texttt{C[k]} = C(n+k)$ and $\texttt{S[k]} = S(n+k)$. We do that with the class `DifferencePolynomialRing`:

In [10]:
DPR.<C,S> = DifferencePolynomialRing(DR)
DPR

Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In [11]:
C, S

(C_*, S_*)

At this point we can create a Difference system with the appropiate equations:

In [5]:
system = DifferenceSystem([
    C[1] - K_se*I_0 * S[0] - (1 - K_cp - K_rb) * C[0] + K_se*S[0]*C[0],
    S[1] - (1 - K_se * I_0) * S[0] - K_rb * C[0] - K_sc * C[0] * S[0]
], variables = [C,S])
system

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

Let us consider now the output variables, that will be another system only focused on $S(n)$, so it will have coefficients using some shifts of $C(n)$:

In [6]:
system_on_S = system.change_variables([S])
system_on_S

System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

And what Sonia checked by hand is that the system formed by $(f_1, \sigma f_1, f_2)$ is a nice system to delete $S(n)$:

In [7]:
extended_system = system_on_S.extend_by_difference([1,0])
print(f"Can we eliminate {extended_system.variables}? --> {extended_system.is_sp2()}")
extended_system

Can we eliminate (S_*,)? --> True


System over [Ring of difference polynomials in (C, S) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(S_*,)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	K_se*C_1*S_1 + C_2 + (K_cp + K_rb - 1)*C_1 + (-I_0*K_se)*S_1 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
}

In [8]:
extended_system.diff_resultant(alg_res="macaulay")

K_se*K_sc*C_1^2*C_0 + (-K_se^2*K_rb + K_se*K_cp*K_sc + K_se*K_rb*K_sc - K_se*K_sc)*C_1*C_0^2 + (-K_se)*C_2*C_0 + (-I_0*K_se^2 + K_se)*C_1^2 + (-I_0*K_se^2*K_cp + I_0*K_se^2 - I_0*K_se*K_sc)*C_1*C_0 + (I_0*K_se^2*K_rb - I_0*K_se*K_cp*K_sc - I_0*K_se*K_rb*K_sc + I_0*K_se*K_sc)*C_0^2 + I_0*K_se*C_2 + (I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - 2*I_0*K_se)*C_1 + (I_0^2*K_se^2*K_cp - I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + I_0*K_se)*C_0

We can also try to compute this resultant from the original system (since `diff_resultant` extends the system):

In [9]:
system_on_S.diff_resultant(alg_res="macaulay", verbose=True)

Getting the appropriate extension for having a SP2 system...
Trying the extension (0, 0)
Trying the extension (1, 0)
Found the valid extension (1, 0)
Getting the homogenize version of the equations...
Computing the Macaulay resultant...


K_se*K_sc*C_1^2*C_0 + (-K_se^2*K_rb + K_se*K_cp*K_sc + K_se*K_rb*K_sc - K_se*K_sc)*C_1*C_0^2 + (-K_se)*C_2*C_0 + (-I_0*K_se^2 + K_se)*C_1^2 + (-I_0*K_se^2*K_cp + I_0*K_se^2 - I_0*K_se*K_sc)*C_1*C_0 + (I_0*K_se^2*K_rb - I_0*K_se*K_cp*K_sc - I_0*K_se*K_rb*K_sc + I_0*K_se*K_sc)*C_0^2 + I_0*K_se*C_2 + (I_0^2*K_se^2 + I_0*K_se*K_cp + I_0*K_se*K_rb - 2*I_0*K_se)*C_1 + (I_0^2*K_se^2*K_cp - I_0^2*K_se^2 - I_0*K_se*K_cp - I_0*K_se*K_rb + I_0*K_se)*C_0

In [10]:
res_y_is_c = system_on_S.diff_resultant(alg_res="macaulay")
coefficients_y_is_c = [el.wrapped for el in res_y_is_c.coefficients()]

In [11]:
with open("/home/anton/results/coefficients_y=c.txt", "w") as f:
    for coeff in coefficients_y_is_c:
        f.write(str(coeff) + "\n")
with open("/home/anton/results/resultant_y=c.txt", "w") as f:
    f.write(str(res_y_is_c))

FileNotFoundError: [Errno 2] No such file or directory: '/home/anton/results/coefficients_y=c.txt'

### Approach with output variables

In the previous example, we kind of cheated, since we took ans an output variable one of the variables that were already in the system. More generically, we will take a new function $Y(n)$ as an output and this will be the function we need to preserve throughout the elimination.

For example, consider te previous system with output variable $Y(n) = S(n) + C(n)$. This will be represented as follows:

In [3]:
DPR.<C,S,Y> = DifferencePolynomialRing(DR)
DPR

Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]

In [4]:
system = DifferenceSystem([
    C[1] - K_se*I_0 * S[0] - (1 - K_cp - K_rb) * C[0] + K_se*S[0]*C[0],
    S[1] - (1 - K_se * I_0) * S[0] - K_rb * C[0] - K_sc * C[0] * S[0],
    Y[0] - C[0] - S[0]
], variables = [C,S])
system

System over [Ring of difference polynomials in (C, S, Y) over Difference Ring [Multivariate Polynomial Ring in I_0, K_se, K_cp, K_rb, K_sc over Rational Field] with difference [Map from callable <lambda>]] with variables [(C_*, S_*)]:
{
	K_se*C_0*S_0 + C_1 + (K_cp + K_rb - 1)*C_0 + (-I_0*K_se)*S_0 == 0
	(-K_sc)*C_0*S_0 + (-K_rb)*C_0 + S_1 + (I_0*K_se - 1)*S_0 == 0
	-C_0 - S_0 + Y_0 == 0
}

From [enhancement #14](https://github.com/Antonio-JP/dalgebra/issues/14), the method `diff_resultant` computes automatically the best algorithms. For linear cases it will use Macaulay resultant, while for non-linear variables it uses and iterative approach to remove variables:

In [5]:
%%time
res = system.diff_resultant()

CPU times: user 537 ms, sys: 8.87 ms, total: 546 ms
Wall time: 544 ms


We can try to apply the elimnation here right away:

In [6]:
res.order(Y), res.degree(), len(res.monomials())

(2, 4, 21)

In [7]:
for coeff in res.coefficients():
    print(f"(deg: {coeff.wrapped.degree()},\tn_mons: {len(coeff.wrapped.monomials())})")

(deg: 6,	n_mons: 3)
(deg: 8,	n_mons: 15)
(deg: 10,	n_mons: 47)
(deg: 11,	n_mons: 79)
(deg: 12,	n_mons: 84)
(deg: 5,	n_mons: 4)
(deg: 7,	n_mons: 22)
(deg: 9,	n_mons: 41)
(deg: 9,	n_mons: 54)
(deg: 11,	n_mons: 104)
(deg: 12,	n_mons: 159)
(deg: 13,	n_mons: 173)
(deg: 4,	n_mons: 5)
(deg: 8,	n_mons: 49)
(deg: 8,	n_mons: 70)
(deg: 12,	n_mons: 161)
(deg: 13,	n_mons: 214)
(deg: 14,	n_mons: 241)
(deg: 9,	n_mons: 31)
(deg: 11,	n_mons: 70)
(deg: 12,	n_mons: 90)


In [141]:
with open("/home/anton/results/coefficients_c+s.txt", "w") as f:
    for coeff in res.coefficients():
        f.write(str(coeff) + "\n")
with open("/home/anton/results/resultant_c+s.txt", "w") as f:
    f.write(str(res))